In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import sys
sys.path.insert(0, "../../code-previous")

In [3]:
import pandas as pd
import numpy as np
import operator

import string
import nltk
stopwords = nltk.corpus.stopwords.words('english')

import pickle
from sklearn.feature_extraction.text import CountVectorizer

import plotly
import plotly.plotly as py
from plotly import tools

import plotly.graph_objs as go
plotly.tools.set_credentials_file(username='sreejith_s', api_key='rcet0rTqFCc3WeOzWTjv')

from unicodedata import category
from tqdm import tqdm
tqdm.pandas(desc="Progress")

import seaborn as sns
sns.set_style('white')
import matplotlib.pylab as plt

/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [4]:
data_path = "/mnt/disks/vault/analysis-data/interdisciplinary-novelty-analysis/all_discipline_data.pql"
data = pd.read_pickle(data_path)

In [5]:
len(data)

406341980

In [6]:
data_update = data[["Label","PubYear","Doctypes", "title_without_stopwords"]]

In [7]:
data_update = data_update.drop_duplicates()

In [8]:
len(data)

406341980

In [9]:
disciplines = ["Architecture","Art","Humanities, Multidisciplinary","Folklore","Medieval & Renaissance Studies","Asian Studies","Classics","Dance","Film, Radio, Television","History","History & Philosophy of Science","Literary Theory & Criticism","Literary Reviews","Literature","Literature, African, Australian, Canadian","Literature, American","Literature, British Isles","Literature, German, Dutch, Scandinavian","Literature, Romance","Literature, Slavic","Poetry","Music","Philosophy","Religion","Theater","Agriculture, Dairy & Animal Science","Agricultural Engineering","Agricultural Economics & Policy","Agriculture, Multidisciplinary","Agronomy","Horticulture","Soil Science","Allergy","Anatomy & Morphology","Anesthesiology","Anthropology","Audiology & Speech-Language Pathology","Behavioral Sciences","Biochemical Research Methods","Biochemistry & Molecular Biology","Biodiversity Conservation","Biophysics","Biotechnology & Applied Microbiology","Cardiac & Cardiovascular Systems","Peripheral Vascular Disease","Cell & Tissue Engineering","Cell Biology","Dentistry, Oral Surgery & Medicine","Dermatology","Developmental Biology","Emergency Medicine","Andrology","Endocrinology & Metabolism","Entomology","Ecology","Environmental Sciences","Environmental Studies","Evolutionary Biology","Fisheries","Food Science & Technology","Forestry","Gastroenterology & Hepatology","Critical Care Medicine","Primary Health Care","Medicine, General & Internal","Genetics & Heredity","Geriatrics & Gerontology","Gerontology","Health Care Sciences & Services","Health Policy & Services","Hematology","Immunology","Infectious Diseases","Integrative & Complementary Medicine","Medicine, Legal","Biology","Limnology","Marine & Freshwater Biology","Mathematical & Computational Biology","Medical Ethics","Medical Informatics","Medical Laboratory Technology","Microbiology","Mycology","Clinical Neurology","Neurosciences","Neuroimaging","Nursing","Nutrition & Dietetics","Obstetrics & Gynecology","Oncology","Ophthalmology","Orthopedics","Otorhinolaryngology","Paleontology","Parasitology","Pathology","Pediatrics","Chemistry, Medicinal","Pharmacology & Pharmacy","Physiology","Plant Sciences","Psychiatry","Public, Environmental & Occupational Health","Radiology, Nuclear Medicine & Medical Imaging","Rehabilitation","Reproductive Biology","Medicine, Research & Experimental","Respiratory System","Rheumatology","Sport Sciences","Substance Abuse","Surgery","Toxicology","Transplantation","Tropical Medicine","Urology & Nephrology","Veterinary Sciences","Virology","Ornithology","Zoology","Astronomy & Astrophysics","Chemistry, Applied","Chemistry, Multidisciplinary","Chemistry, Analytical","Chemistry, Inorganic & Nuclear","Chemistry, Organic","Chemistry, Physical","Crystallography","Electrochemistry","Geochemistry & Geophysics","Geology","Geosciences, Multidisciplinary","Mathematics, Applied","Mathematics, Interdisciplinary Applications","Mathematics","Statistics & Probability","Meteorology & Atmospheric Sciences","Mineralogy","Mining & Mineral Processing","Oceanography","Optics","Geography, Physical","Physics, Applied","Physics, Fluids & Plasmas","Physics, Atomic, Molecular & Chemical","Physics, Multidisciplinary","Physics, Condensed Matter","Physics, Nuclear","Physics, Particles & Fields","Physics, Mathematical","Polymer Science","Thermodynamics","Water Resources","Nanoscience & Nanotechnology","Logic","Multidisciplinary Sciences","Acoustics","Automation & Control Systems","Computer Science, Artificial Intelligence","Computer Science, Cybernetics","Computer Science, Hardware & Architecture","Computer Science, Information Systems","Computer Science, Interdisciplinary Applications","Computer Science, Software Engineering","Computer Science, Theory & Methods","Construction & Building Technology","Energy & Fuels","Engineering, Aerospace","Engineering, Multidisciplinary","Engineering, Biomedical","Engineering, Environmental","Engineering, Chemical","Engineering, Industrial","Engineering, Manufacturing","Engineering, Marine","Engineering, Civil","Engineering, Ocean","Engineering, Petroleum","Engineering, Electrical & Electronic","Engineering, Mechanical","Engineering, Geological","Ergonomics","Imaging Science & Photographic Technology","Information Science & Library Science","Instruments & Instrumentation","Materials Science, Paper & Wood","Materials Science, Ceramics","Materials Science, Multidisciplinary","Materials Science, Biomaterials","Materials Science, Characterization & Testing","Materials Science, Coatings & Films","Materials Science, Composites","Materials Science, Textiles","Mechanics","Metallurgy & Metallurgical Engineering","Microscopy","Nuclear Science & Technology","Operations Research & Management Science","Remote Sensing","Robotics","Spectroscopy","Telecommunications","Transportation","Transportation Science & Technology","Archaeology","Area Studies","Social Sciences, Biomedical","Business","Business, Finance","Economics","Industrial Relations & Labor","Management","Communication","Criminology & Penology","Cultural Studies","Demography","Education & Educational Research","Education, Scientific Disciplines","Education, Special","Ethnic Studies","Family Studies","Geography","Law","Political Science","International Relations","Linguistics","Language & Linguistics","Social Sciences, Mathematical Methods","Psychology, Biological","Psychology, Clinical","Psychology, Educational","Psychology, Developmental","Psychology, Applied","Psychology","Psychology, Multidisciplinary","Psychology, Psychoanalysis","Psychology, Mathematical","Psychology, Experimental","Psychology, Social","Planning & Development","Public Administration","Social Issues","Ethics","History of Social Sciences","Hospitality, Leisure, Sport & Tourism","Social Sciences, Interdisciplinary","Social Work","Sociology","Urban Studies","Women's Studies"]

In [10]:
pd.set_option('display.max_colwidth', -1)

In [11]:
data_update.head(20)

,Label,PubYear,Doctypes,title_without_stopwords
0,Surgery,1981,Article,craniofacial surgical techniques aesthetic results congenital acute traumatic deformities
2,Life Sciences & Biomedicine,1981,Article,craniofacial surgical techniques aesthetic results congenital acute traumatic deformities
3,Science & Technology,1981,Article,craniofacial surgical techniques aesthetic results congenital acute traumatic deformities
4,Science & Technology,1981,Letter,lack effect disopyramide chronic stable angina
5,Life Sciences & Biomedicine,1981,Letter,lack effect disopyramide chronic stable angina
6,Cardiac & Cardiovascular Systems,1981,Letter,lack effect disopyramide chronic stable angina
7,Pharmacology & Pharmacy,1981,Letter,lack effect disopyramide chronic stable angina
8,Cardiovascular System & Cardiology,1981,Letter,lack effect disopyramide chronic stable angina
10,Science & Technology,1981,Article,investigation influencing variables when determining permissible stack height storage space
11,Life Sciences & Biomedicine,1981,Article,investigation influencing variables when determining permissible stack height storage space


__plot distribution of the publications wrt disciplines__

In [12]:
data = data_update[data_update['Label'].isin(disciplines)]

In [13]:
len(data)

61860280

In [14]:
# write filtered data to pickle
# filename = "/mnt/disks/vault/analysis-data/word-journey/all-data.pql"
# pickle_out = open(filename,"wb")
# pickle.dump(data, pickle_out)
# pickle_out.close()

In [15]:
len(data.Label.unique())

249

In [16]:
data_grouped_by_label = data[["Label","title_without_stopwords"]].groupby(data.Label)["title_without_stopwords"].apply(list)
data_grouped_by_label = pd.DataFrame(data_grouped_by_label)

In [17]:
len(data_grouped_by_label)

249

In [18]:
data_grouped_by_label["cleaned_titles"] = data_grouped_by_label["title_without_stopwords"].apply(lambda x: list(set(x)))

data_grouped_by_label["before_distinct"] = data_grouped_by_label["title_without_stopwords"].apply(lambda x: len(x))
data_grouped_by_label["after_distinct"] = data_grouped_by_label["cleaned_titles"].apply(lambda x: len(x))

data_grouped_by_label.head()

title_without_stopwords  \
Label                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                

In [19]:
data_grouped_by_label["title_words_"] = data_grouped_by_label["cleaned_titles"].progress_apply(lambda x: " ".join(x))

Progress: 100%|██████████| 249/249 [00:29<00:00,  8.32it/s]


In [20]:
def num_there(w):
    return any(i.isdigit() for i in w)


def clean_digits(s):
    t = s.split()
    cleaned = " ".join([x for x in t if not num_there(x)])
    return cleaned

data_grouped_by_label["title_words"] = data_grouped_by_label["title_words_"].progress_apply(lambda x: clean_digits(x))

Progress: 100%|██████████| 249/249 [14:15<00:00,  2.41s/it]


In [29]:
# write filtered data to pickle
filename = "/mnt/disks/vault/analysis-data/interdisciplinary-novelty-analysis/corrected-entropy-analysis/all-data-grouped.pql"
pickle_out = open(filename,"wb")
pickle.dump(data_grouped_by_label, pickle_out)
pickle_out.close()

In [21]:
vectorizer = CountVectorizer()
vectorized_words = vectorizer.fit_transform(data_grouped_by_label.title_words)

In [31]:
pickle_out = open("/mnt/disks/vault/analysis-data/interdisciplinary-novelty-analysis/corrected-entropy-analysis/vectorizer.pickle","wb")
pickle.dump(vectorizer, pickle_out)
pickle_out.close()

pickle_out = open("/mnt/disks/vault/analysis-data/interdisciplinary-novelty-analysis/corrected-entropy-analysis/vectorized_words.pickle","wb")
pickle.dump(vectorized_words, pickle_out)
pickle_out.close()

In [22]:
count_vocab = {value:key for key,value in vectorizer.vocabulary_.items()}
count_rev_dict = {value:key for key,value in vectorizer.vocabulary_.items()}
words = [count_rev_dict[i] for i in range(len(vectorizer.vocabulary_.keys()))]

In [23]:
vectorized_wordarray = vectorized_words.toarray()

In [24]:
document_frequency = (vectorized_wordarray != 0).sum(0).tolist()
document_frequency_log =  [np.log(x) for x in document_frequency]
#2. Term Frequency - Number of times in which a word appears in a document
term_frequency = vectorized_words.sum(axis=0).reshape(-1,).tolist()[0]
term_frequency_log = [np.log(x) for x in term_frequency]

In [31]:
trace0 = go.Scattergl(
    x = document_frequency_log[:100000],
    y = term_frequency_log[:100000],
    mode = 'markers',
    text = words[:100000],
    marker = dict(
        color = '#FFBAD2',
        line = dict(width = 1)
    )
)


layout= go.Layout(
    title= 'Document Frequency - Term Frequency plot ',
    hovermode= 'closest',
    xaxis= dict(
        title= 'Log Document Frequency (Number of disciplines in which the word occurs)',
        ticklen= 5,
        zeroline= False,
        gridwidth= 2,
    ),
    yaxis=dict(
        title= 'Log Term Frequency (Count of the word in corpus)',
        ticklen= 5,
        gridwidth= 2,
    ),
    
    showlegend= False
)

plotdata = [trace0]


fig= go.Figure(data=plotdata, layout=layout)
py.iplot(fig)

/usr/local/lib/python3.6/dist-packages/IPython/core/display.py:689: UserWarning:

Consider using IPython.display.IFrame instead



__Pick some words which have low log document frequency and relatively high term frequency__

_(i,e words that appear in only a few disciplines but have a good frequency count)_


In [40]:
len(np.unique(term_frequency_log))

16738

In [58]:
good_words = [words[index] for index in range(len(term_frequency_log)) if term_frequency_log[index] > 6.5 and document_frequency_log[index] < 2.5]

In [59]:
len(good_words)

127

In [60]:
good_words

['2mass',
 'andromedae',
 'anosov',
 'antiquark',
 'artinian',
 'aurigae',
 'auslander',
 'baryogenesis',
 'baryonic',
 'beurling',
 'birational',
 'blazar',
 'blazars',
 'branes',
 'braneworld',
 'brst',
 'cabibbo',
 'calabi',
 'capsulorhexis',
 'cassiopeiae',
 'cepheids',
 'chevalley',
 'cobordism',
 'cocycles',
 'compactified',
 'dilatonic',
 'dilepton',
 'diphoton',
 'diquark',
 'disolvate',
 'eclogites',
 'efirov',
 'eiscat',
 'ethyleniques',
 'fastbus',
 'floer',
 'glueball',
 'grassmannians',
 'gravitino',
 'harnack',
 'hecke',
 'heegaard',
 'hemisolvate',
 'herbig',
 'hinode',
 'horava',
 'hyades',
 'hyers',
 'hypoelliptic',
 'inflaton',
 'interlibrary',
 'iridotomy',
 'iwasawa',
 'izuchenie',
 'jona',
 'kislot',
 'kisloty',
 'koszul',
 'krull',
 'lasinio',
 'lectotypification',
 'leptogenesis',
 'lusztig',
 'markarian',
 'metabelian',
 'metapelites',
 'motivic',
 'nannofossil',
 'neutralino',
 'nonleptonic',
 'ogle',
 'ophiuchi',
 'opredeleniya',
 'orbifold',
 'orionis',
 'os3